In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [2]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")


sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [3]:
DATA_VERSION = "v001"
TRIAL_NO = "012"

In [4]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.shape

(2505542, 2)

In [5]:
n_sum = 0
sub_split_list = []
for mol_type in [1,2,3,4]:
    test_type_cut = np.load(f"../processed/v002/test_type{mol_type}_cut.npy", )
    print(test_type_cut.sum())
    n_sum += test_type_cut.sum()
    sub_split_list.append(sub[test_type_cut])
sub_split_df = pd.concat(sub_split_list)
n_sum

380609
24195
203126
1897612


2505542

In [6]:
sub_split_df.shape

(2505542, 2)

In [7]:
sub_split_df.head()

,id,scalar_coupling_constant
1,4658148,0
3,4658150,0
5,4658152,0
9,4658156,0
12,4658159,0


In [8]:
sub_files = np.sort(glob(f"../submit/{DATA_VERSION}_{TRIAL_NO}/submission_*.csv"))

In [9]:
ls -l ../submit

total 28
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  7 15:53 v001_001/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  7 15:53 v001_002/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  7 15:53 v001_004/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  7 15:53 v001_005/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  7 15:53 v001_006/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  8 22:41 v001_009/
drwxrwxr-x 2 kenichi.matsui kenichi.matsui 4096 Jun  9 10:31 v001_012/


In [10]:
ls -l ../submit/v001_012

total 166800
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 18334305 Jun  9 00:46 oof_1.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  1121191 Jun  9 01:17 oof_2.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 10326335 Jun  9 03:25 oof_3.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 93917364 Jun  9 09:50 oof_4.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  6883224 Jun  9 00:56 submission_1.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui   440442 Jun  9 01:17 submission_2.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  3972902 Jun  9 03:32 submission_3.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 35789840 Jun  9 10:31 submission_4.csv


In [11]:
sub_files

array(['../submit/v001_012/submission_1.csv',
       '../submit/v001_012/submission_2.csv',
       '../submit/v001_012/submission_3.csv',
       '../submit/v001_012/submission_4.csv'], dtype='<U35')

In [12]:
!head ../submit/v001_009/submission_1.csv

scalar_coupling_constant
156.70772963513656
149.98357017582478
91.04887956561781
91.96889418882229
81.15795546727574
89.65331088741222
90.08662751050795
82.75465992839082
109.9355144772342


In [13]:
sub_list = []
for f in sub_files:
    df_ = pd.read_csv(f)
    print(f"df.shape: {df_.shape}")
    sub_list += [df_]
    
sub_df = pd.concat(sub_list, axis=0)

df.shape: (380609, 1)
df.shape: (24195, 1)
df.shape: (203126, 1)
df.shape: (1897612, 1)


In [ ]:
# 1: 709416, 2: 43363, 3: 378036, 4: 3527332
# (380609, 24195, 203126, 1897612)

In [14]:
sub_df.shape

(2505542, 1)

In [15]:
sub_df.head()

,scalar_coupling_constant
0,160.96248
1,151.41955
2,91.11808
3,91.95610
4,81.71851


In [16]:
sub_split_df.head()

,id,scalar_coupling_constant
1,4658148,0
3,4658150,0
5,4658152,0
9,4658156,0
12,4658159,0


In [17]:
sub_split_df["scalar_coupling_constant"] = sub_df["scalar_coupling_constant"].values

In [18]:
sub_split_df.head()

,id,scalar_coupling_constant
1,4658148,160.96248
3,4658150,151.41955
5,4658152,91.11808
9,4658156,91.95610
12,4658159,81.71851


In [19]:
f"../submit/{DATA_VERSION}_{TRIAL_NO}/submit_all_{DATA_VERSION}_{TRIAL_NO}.csv"

'../submit/v001_012/submit_all_v001_012.csv'

In [20]:
sub_split_df.to_csv(f"../submit/{DATA_VERSION}_{TRIAL_NO}/submit_all_{DATA_VERSION}_{TRIAL_NO}.csv", index=False)

In [21]:
ls -l ../submit/v001_012

total 232380
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 18334305 Jun  9 00:46 oof_1.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  1121191 Jun  9 01:17 oof_2.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 10326335 Jun  9 03:25 oof_3.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 93917364 Jun  9 09:50 oof_4.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  6883224 Jun  9 00:56 submission_1.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui   440442 Jun  9 01:17 submission_2.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui  3972902 Jun  9 03:32 submission_3.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 35789840 Jun  9 10:31 submission_4.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 67151306 Jun  9 11:02 submit_all_v001_012.csv


In [23]:
# !kaggle competitions submit -c champs-scalar-coupling -f '../submit/v001_012/submit_all_v001_012.csv' -m "split 4 parts + angle"

100%|██████████████████████████████████████| 64.0M/64.0M [00:03<00:00, 21.5MB/s]
Successfully submitted to Predicting Molecular Properties